# __[:+:] -- DragonVision Prototype-- [:+:]__

In [4]:
# [+] Libraries that are commented out are only for ROS
#import rospy
#from std_msgs.msg import Float64
#from sensor_msgs.msg import Image
#from cv_bridge import CvBridge, CvBridgeError
import numpy as np
import cv2
import math
import time

#[+------:
import sys

bgrRed= (0, 0, 255)
bgrBlue= (255, 0, 0)
bgrCyan= (200, 200, 50)
bgrOrange= (100, 100, 255)
bgrYellow= (75, 200, 255)


## __[::] Gate (2022)__ 

In [10]:
current_yaw = 0
cv_image = 0
YAW_VARIANCE = .017

# [+]- Access the camera, the int parameter determines which camera you are using, may have to change in depending on what computer you run on
video_capture = cv2.VideoCapture(0)
# video_capture = cv2.VideoCapture(1)
# video_capture = cv2.VideoCapture(2)

#[+]- Each iteration of this loop processes a frame that is captured by the camera by applying a series of filters. Each filter is in intermediate step, the final image is the one which we annotate and extract information from. 
while(True):
    # Capture the frames
    ret, frame = video_capture.read()
    final = frame

    # change from bgr values to hsv values
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # color bounds of color to be filtered out [+] correction: colors to be NOT filtered out
    # lower_color_bounds = np.array([0, 200, 0])
    # upper_color_bounds = np.array([255, 255, 255])
    lower_color_bounds = np.array([0, 100, 20])
    upper_color_bounds = np.array([30, 255, 255])


    # [+]- Filter 1: threshold shows in black the pixels being filtered out [+: show only the colors that are between the two bounds)
    threshold = cv2.inRange(hsv, lower_color_bounds, upper_color_bounds)

    #[+]- Filter 2:  erode to remove noise
    kernel = np.ones((10, 10), np.uint8)
    erode = cv2.erode(threshold, kernel)

    # get the contours, Each individual contour is a Numpy array of (x,y) coordinates of boundary points of the object
    contours = cv2.findContours(erode, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[0]

    # create bounding boxes, creates an array of contour areas so that we can know which is th ebiggest
    contour_areas = [cv2.contourArea(x) for x in contours]  
    
    # get the areas of each contour, returns a list of indeces for contour areas array
    contour_indexes = np.argsort(contour_areas)  
    
    # sort the indexes of the largest areas
    for i in contour_indexes[-2:]:  # only look at the two largest contours
        (x,y,w,h) = cv2.boundingRect(contours[i])  # get the location/dimensions of a bounding box for the contour: x,y=coordinates, w,h=dims
        cv2.rectangle(final, pt1=(x,y), pt2=(x+w,y+h), color=(255,0,0), thickness=5)  # draw the bounding box on the image

        # for visibility, we will place a background fill on the contour label
        text = "gatepost"
        text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_PLAIN, 1, 1)
        text_w, text_h = text_size
        cv2.rectangle(final, pt1=(x, y), pt2=(x + text_w, y - 2*text_h), color=(255, 0, 0), thickness=-1)
        cv2.putText(final, "gatepost", org=(x, y-5), fontFace=cv2.FONT_HERSHEY_PLAIN, fontScale=1, color=(255, 255, 255), thickness=1)

    # [+]- Show each filter that is applied to the frame. Each filter is shown in a separate window
    # cv2.imshow('1 Gate: original frame',frame)
    cv2.imshow('2 Gate: threshold', threshold)
    cv2.imshow("3 Gate: eroded", erode)
    cv2.imshow("4 Gate: final", final)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

## __[::] Buoy (2022)__ 

In [1]:
current_yaw = 0
cv_image = 0
YAW_VARIANCE = .017

# [+]- Access the camera, the int parameter determines which camera you are using, may have to change in depending on what computer you run on
video_capture = cv2.VideoCapture(0)
# video_capture = cv2.VideoCapture(1)
# video_capture = cv2.VideoCapture(2)

# # color bounds of color to be [NOT] filtered out, by default it filters out light colors so that it can identif 
lower_value_bounds = np.array([0, 0, 0])
upper_value_bounds = np.array([255, 255, 45])


#[+:]-- -------------------------------------------------//
lower_value_bounds2 = np.array([0, 0, 0])
upper_value_bounds2 = np.array([255, 255, 80])
clk= 0
clkRate= 9
clkLim= clkRate*255
#-------------------------------------------------------//

dilateKernel = np.ones((5, 5), np.uint8)
erodeKernel = np.ones((10, 10), np.uint8)
erodeKernel2= np.ones((6, 6), np.uint8)
blurnel = (5, 5)


while(True):
    # Capture the frames
    ret, img_frame = video_capture.read()
    img_final = img_frame

    # change from bgr values to hsv values
    img_hsv = cv2.cvtColor(img_frame, cv2.COLOR_BGR2HSV)
    

    # threshold shows in black the pixels being filtered out
    img_threshold = cv2.inRange(img_hsv, lower_value_bounds, upper_value_bounds)

    #  [+:] -- Use this to find the ideal value range, which changes depending on the lighting ----------//
    tick= clk//clkRate
    # img_threshold = cv2.inRange(img_hsv, np.array([0, 0, 0]), np.array([255, 255, tick]))
    # cv2.putText(img_final, "[:+:] CLK: "+ str(tick), org=(10, 20), fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale=.7, color=(200, 150, 0), thickness=2)
    # clk+= 1
    # if clk>clkLim: clk=0
    #-----------------------------------------------------------------------------------------------------//

    # dilate to combine contours
    img_dilate = cv2.dilate(img_threshold, dilateKernel, iterations=3)

    # erode to remove noise
    img_erode = cv2.erode(img_dilate, erodeKernel, iterations=2)

    # blur to smooth edges so circle detection is easier
    img_blur = cv2.blur(img_erode, ksize= blurnel)

    # detect circles
    circles = cv2.HoughCircles(img_blur, cv2.HOUGH_GRADIENT, 1, 20,
                  param1=30,  # edge detection parameter
                  param2=30,  # accumulator threshold, or how circley the circles need to be to be recognized (higher=more circlely)
                  minRadius=0,
                  maxRadius=100)

    # if a circle is identified
    if (type(circles)) is np.ndarray:
        # draw the circles
        circle_radii = [x[2] for x in circles[0]]  # get the radii of each contour
        circle_indexes = np.argsort(circle_radii)  # sort by largest radius
        for i in circle_indexes[-2:]:  # only contour at the largest circles
            circle = circles[0][i]  # get the largest circle
            cv2.circle(img_final, center=(int(circle[0]), int(circle[1])), radius=int(circle[2]), color=(100, 0, 255), thickness=2)  # draw the circle on the image
            # make the text centered
            # text = "police buoy"
            text = " --  -- "
            text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_PLAIN, 1, 1)  # get the text size
            text_w, text_h = text_size  # get the text width/height
            cv2.putText(img_final, text, org=(int(circle[0])-text_w, int(circle[1])+(text_h//2)+8), fontFace=cv2.FONT_HERSHEY_PLAIN, fontScale=2, color=(100, 0, 255), thickness=2)


    #[:+:]===============================================================================================================================================[:+:]
    clkRate= 1
    #[+]-------- ----------------------------------------------------------------------------//
    img_threshold2 = cv2.inRange(img_hsv, np.array([0, 0, 0]), np.array([255, 255, 80]))
    cv2.putText(img_final, ("[:+:] CLK: "+ str(tick)), org=(10, 20), fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale=.7, color=(100, 255, 100), thickness=2)
    clk+= 1
    if clk>clkLim: clk=0
    #----------------------------------------------------------------------------------------//

    img_erode2 = cv2.erode(img_threshold2, erodeKernel2, iterations=1)

    dilateKernel2 = np.ones((5, 5), np.uint8)

    img_dilate2 = cv2.dilate(img_erode2, dilateKernel2, iterations=3)

    img_blur2 = cv2.blur(img_erode2, ksize=blurnel)

    circles2 = cv2.HoughCircles(img_blur2, cv2.HOUGH_GRADIENT, 1, 20,
                  param1=30,  # edge detection parameter
                  param2=30,  # accumulator threshold, or how circley the circles need to be to be recognized (higher=more circlely)
                  minRadius=0,
                  maxRadius=100)

#[+:] --- Circle Array # 2 -----------------------------------------
    if (type(circles2)) is np.ndarray:
        circle_radii = [x[2] for x in circles2[0]]  
        circle_indexes = np.argsort(circle_radii)  
        for i in circle_indexes[-2:]:  
            circle = circles2[0][i] 
            cv2.circle(img_final, center=(int(circle[0]), int(circle[1])), radius=int(circle[2])+25, color=(200, 150, 0), thickness=2) 
            text = "[:+:]"
            text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_DUPLEX, 1, 1)  
            text_w, text_h = text_size  
            cv2.putText(img_final, text, org=(int(circle[0])-text_w, int(circle[1])+text_h), fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale=2, color=(200, 150, 0), thickness=2)
#[:\:]===============================================================================================================================================[:\:]



    # [+] show frames
    # cv2.imshow('1 original frame', img_frame)
    # cv2.imshow('2 Buoy: hsv', img_hsv)
    # cv2.imshow('3 Buoy: threshold', img_threshold)
    # cv2.imshow("4 Buoy: dilated", img_dilate)
    # cv2.imshow("5 Buoy: eroded", img_erode)
    # cv2.imshow("6 Buoy: blur", img_blur)
    # cv2.imshow("7 Buoy: final", img_final)

    #[+: ------------------------------------------------- ::
    # cv2.imshow('1 original frame2', img_frame)
    # cv2.imshow('2 Buoy: hsv2', img_hsv)
    cv2.imshow('3 Buoy: threshold2', img_threshold2)
    cv2.imshow("4 Buoy: dilated2", img_dilate2)
    cv2.imshow("5 Buoy: eroded2", img_erode2)
    cv2.imshow("6 Buoy: blur2", img_blur2)
    #::-----------------------------------------------------:+]
    cv2.imshow("7 Buoy: final2", img_final)

  
  

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()


NameError: name 'cv2' is not defined

## __[::] Circle Detection__ 

In [2]:
import numpy as np
import cv2

current_yaw = 0
cv_image = 0
YAW_VARIANCE = .017

# [+]- Access the camera, the int parameter determines which camera you are using, may have to change in depending on what computer you run on
video_capture = cv2.VideoCapture(0)
# video_capture = cv2.VideoCapture(1)
# video_capture = cv2.VideoCapture(2)

lower_value_bounds = np.array([0, 0, 0])
upper_value_bounds = np.array([255, 255, 75])

houghParam1=40
houghParam2=45
# houghParam1=15
# houghParam2=30

#[+:]-- -------------------------------------------------//
clk= 0
clkRate= 1
clkLim= clkRate*255
#-------------------------------------------------------//


kernel_sqr = np.ones((1,1), np.uint8)
kernel_3x1= np.ones((3,1), np.uint8)
kernel_1x3= np.ones((1,3), np.uint8)
blurnel = (2, 2)
while(True):
    ret, img_frame = video_capture.read()
    img_final = img_frame

    # [+:]--- Use this to find the ideal value range, which changes depending on the lighting ---------------------//
    tick= clk//clkRate + 1
    cv2.putText(img_final, "[:+:] CLK: "+ str(tick), org=(10, 20), fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale=.7, color=(200, 0, 255), thickness=2)
    clk+= 1
    if clk>clkLim: clk=0
    #--------------------------------------------------------------------------------------------------------------//

    # #[::+::]======================Debug================|+|+|+|
    # clkRate= 1
    # upper_value_bounds = np.array([255, 255, 75])
    # # [\\+\\]===========================================|-|-|-|


    img_hsv = cv2.cvtColor(img_frame, cv2.COLOR_BGR2HSV)
    img_threshold = cv2.inRange(img_hsv, lower_value_bounds, upper_value_bounds)
    img_dilate = cv2.dilate(img_threshold, kernel_1x3, iterations=4)
    img_erode = cv2.erode(img_dilate, kernel_1x3, iterations=4)
    img_blur = cv2.blur(img_erode, ksize= blurnel)

    circles = cv2.HoughCircles(img_blur, cv2.HOUGH_GRADIENT, 1, 20,
                  param1=houghParam1,  # edge detection parameter
                  param2=houghParam2,  # accumulator threshold, or how circley the circles need to be to be recognized (higher=more circlely)
                  minRadius=0,
                  maxRadius=100)

    if (type(circles)) is np.ndarray:
        circle_radii = [x[2] for x in circles[0]] 
        circle_indexes = np.argsort(circle_radii)  
        for i in circle_indexes[-2:]: 
            circle = circles[0][i] 
            cv2.circle(img_final, center=(int(circle[0]), int(circle[1])), radius=int(circle[2]+20), color=bgrOrange, thickness=2)  
            text = " -- -- "
            text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_PLAIN, 1, 1)  # get the text size
            text_w, text_h = text_size  # get the text width/height
            cv2.putText(img_final, text, org=(int(circle[0])-text_w, int(circle[1])+(text_h//2)+7), fontFace=cv2.FONT_HERSHEY_PLAIN, fontScale=2, color=bgrOrange, thickness=2)
            
    # cv2.imshow('2 Circle_1: hsv', img_hsv)
    cv2.imshow('3 Circle_1: threshold', img_threshold)
    cv2.imshow("4 Circle1_: dilated", img_dilate)
    cv2.imshow("5 Circle_1: eroded", img_erode)
    cv2.imshow("6 Circle_1: blur", img_blur)
    cv2.imshow("7 Circle_1: final", img_final)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

## __[::] Circle Detection 2__  

In [8]:
import cv2
import numpy as np

current_yaw = 0
cv_image = 0
YAW_VARIANCE = .017

# [+]- Access the camera, the int parameter determines which camera you are using, may have to change in depending on what computer you run on
video_capture = cv2.VideoCapture(0)
# video_capture = cv2.VideoCapture(1)
# video_capture = cv2.VideoCapture(2)

lower_value_bounds = np.array([0, 0, 150])
# lower_value_bounds = np.array([0, 0, 165])
# lower_value_bounds = np.array([0, 0, 180])
upper_value_bounds = np.array([255, 255, 255])

# houghParam1=40
# houghParam2=45
houghParam1=15
houghParam2=30

clk= 0
clkRate= 1
clkLim= clkRate*255



kernel_sqr2 = np.ones((2,2), np.uint8)
kernel_sqr3 = np.ones((3,3), np.uint8)
kernel_sqr4 = np.ones((4,4), np.uint8)
kernel_sqr5 = np.ones((5,5), np.uint8)
kernel_sqr6 = np.ones((6,6), np.uint8)
kernel_sqr7 = np.ones((7,7), np.uint8)
kernel_sqr8 = np.ones((8,8), np.uint8)
kernel_sqr9 = np.ones((9,9), np.uint8)
kernel_sqr10 = np.ones((10,10), np.uint8)
kernel_sqr11 = np.ones((11,11), np.uint8)
kernel_sqr12 = np.ones((12,12), np.uint8)
kernel_sqr13 = np.ones((13,13), np.uint8)
kernel_sqr13 = np.ones((14,14), np.uint8)
blurnel = (5, 5)
kernel_3x1= np.ones((3,1), np.uint8)
kernel_1x3= np.ones((1,3), np.uint8)
kernel_2x1= np.ones((2,1), np.uint8)
kernel_sqr= np.empty((0,), dtype= np.ndarray)
erodeKernel= kernel_sqr8

while(True):
    ret, img_frame = video_capture.read()
    img_final = img_frame

    # [+:]--- Use this to find the ideal value range, which changes depending on the lighting ---------------------//
    tick= clk//clkRate + 1
    cv2.putText(img_final, str(tick), org=(10, 20), fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale=.5, color= bgrOrange, thickness=2)
    clk+= 1
    if clk>clkLim: clk=0
    #--------------------------------------------------------------------------------------------------------------//

#[::+::]==========|============Debug===========|===========================|+|+|+|
    #-- circl 3 starter ----
    # clkRate= 3
    # morphKernel= cv2.getStructuringElement(cv2.MORPH_CROSS, (2,2))  
    # morphKernel2= cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))  
    # blurnel2= (3, 3)
    # morphDilate_i= 1
    # img_blur = cv2.blur(img_morphDilate, ksize= blurnel2)
    # img_morphDilate = cv2.dilate(img_morph,morphKernel2, iterations=morphDilate_i)
    # morphKernel= np.ones((2,2), np.uint8)
    # img_morph= cv2.morphologyEx(img_threshold, cv2.MORPH_GRADIENT, morphKernel)

    # cv2.imshow('4.5 Circle_2: morph', img_morph)
    # cv2.imshow('4.5 Circle_2: tmp morph->dil', img_morphDilate)
#[\\+\\]==========|============================|===========================|-|-|-|

    img_hsv = cv2.cvtColor(img_frame, cv2.COLOR_BGR2HSV)
    img_threshold = cv2.inRange(img_hsv, lower_value_bounds, upper_value_bounds)
    #[+:]-- erodeKernel= np.ones((7:14,7:14), np.uint8)
    img_erode = cv2.erode(img_threshold, erodeKernel, iterations=1)
    img_dilate = cv2.dilate(img_erode, kernel_sqr6, iterations=1)
    img_blur = cv2.blur(img_dilate, ksize= blurnel)
    


    circles = cv2.HoughCircles(img_blur, cv2.HOUGH_GRADIENT, 1, 20,
                  param1=houghParam1,  # edge detection parameter
                  param2=houghParam2,  
                  minRadius=0,
                  maxRadius=100)

    if (type(circles)) is np.ndarray:
        circle_radii = [x[2] for x in circles[0]] 
        circle_indexes = np.argsort(circle_radii)  
        for i in circle_indexes[-2:]: 
            circle = circles[0][i] 
            cv2.circle(img_final, center=(int(circle[0]), int(circle[1])), radius=int(circle[2]), color=bgrCyan, thickness=2)  
            text = " + "
            text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_PLAIN, 1, 1) 
            text_w, text_h = text_size  
            cv2.putText(img_final, text, org=(int(circle[0]-text_w), int(circle[1])+(text_h//2)+7), fontFace=cv2.FONT_HERSHEY_PLAIN, fontScale=2, color=bgrOrange, thickness=2)
            
    # cv2.imshow('2 Circle_1: hsv', img_hsv)
    cv2.imshow('1 Circle_2: threshold', img_threshold)
    cv2.imshow("2 Circle_2: eroded", img_erode)
    cv2.imshow("4 Circle_2_: dilated", img_dilate)
    cv2.imshow("5 Circle_2: blur", img_blur)
    cv2.imshow("6 Circle_2: final", img_final)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

##  __[:+:] Circle Detection 3 - Morphological Gradient__

In [26]:
import cv2
import numpy as np

current_yaw = 0
cv_image = 0
YAW_VARIANCE = .017

video_capture = cv2.VideoCapture(0)
# video_capture = cv2.VideoCapture(1)
# video_capture = cv2.VideoCapture(2)

clk= 0
clkRate= 1
clkLim= clkRate*255
isDetected= False

lower_value_bounds = np.array([0, 0, 150])
upper_value_bounds = np.array([255, 255, 255])

houghParam1=15
houghParam2=10


morphKernel= cv2.getStructuringElement(cv2.MORPH_CROSS, (2,2))  
morphKernel2= cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))  
blurnel= (3,3)

while(True):
    ret, img_frame = video_capture.read()
    img_final = img_frame



    img_hsv = cv2.cvtColor(img_frame, cv2.COLOR_BGR2HSV)
    img_threshold = cv2.inRange(img_hsv, lower_value_bounds, upper_value_bounds)

    #[+:]-- replace erode and dilate from circ 2 with morph gradient--
    img_morphGrad = cv2.morphologyEx(img_threshold, cv2.MORPH_GRADIENT, morphKernel)
    
    img_dilate = cv2.dilate(img_morphGrad,morphKernel2, iterations=1)
    img_blur = cv2.blur(img_dilate, ksize= blurnel)
    


    circles = cv2.HoughCircles(img_blur, cv2.HOUGH_GRADIENT, 1, 20,
                  param1=houghParam1,  # edge detection parameter
                  param2=houghParam2,  
                  minRadius=0,
                  maxRadius=100)
    
    
    if (type(circles)) is np.ndarray:
        circle_radii = [x[2] for x in circles[0]]  
        circle_indexes = np.argsort(circle_radii)  
        for i in circle_indexes[-2:]:  # only contour at the largest circles
            circle = circles[0][i]  
            cv2.circle(img_final, 
                       center=(int(circle[0]), int(circle[1])), 
                       radius=int(circle[2]), 
                       color=bgrYellow, 
                       thickness=2)  
            text = " (       ) "
            text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_PLAIN, 1, 1)  
            text_w, text_h = text_size     
            origX= int(circle[0])-text_w
            origY= int(circle[1])+(text_h//2)+10                                 
            cv2.putText(img_final, text, 
                org=(origX, origY ), 
                fontFace=cv2.FONT_HERSHEY_PLAIN, 
                fontScale=2, 
                color= bgrYellow, 
                thickness=2)
#[::+::]=======================================|===================Heuristics==================|==================================|+|+|+|\\

    tick= clk//clkRate + 1
    cv2.putText(img_final, str(tick), org=(10, 20), fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale= .7, color= bgrRed, thickness=2)
    clk+= 1
    if clk>clkLim: clk=0

    if (type(circles)) is np.ndarray: 
        cirCnt= circles.shape[1]
    cv2.putText(img_final, str(houghParam2), org=(10, 40), fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale=.5, color= bgrRed, thickness=2)
    cv2.putText(img_final, str(cirCnt), org=(10, 60), fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale=.5, color= bgrRed, thickness=2)
    if cirCnt>2:
        houghParam2+=1
    elif cirCnt<2:
        houghParam2-=1
#[\\+\\]=======================================|==========================================|==================================|-|-|-|//
            
    # cv2.imshow('2 Circle_1: hsv', img_hsv)
    cv2.imshow('1 Circle_3: Threshold', img_threshold)
    cv2.imshow("2 Circle_3: Thresh->Morph", img_morphGrad)
    cv2.imshow("4 Circle_3: Morph->Dilate", img_dilate)
    cv2.imshow("5 Circle_3: Dilate->Blur", img_blur)
    cv2.imshow("6 Circle_3: Final", img_final)



    if cv2.waitKey(1) & 0xFF == ord('q'):
        print(houghParam2)
        print(cirCnt)
        break
cv2.destroyAllWindows()

46
2


##  __[:+:] OCR__

In [6]:
import cv2
import numpy as np

current_yaw = 0
cv_image = 0
YAW_VARIANCE = .017

camNum= 0
video_capture = cv2.VideoCapture(camNum)

clk= 0
clkRate= 1
clkLim= clkRate*255

grayUpp= 255
grayLow= 150
threshUpp= 255
threshLow= 150
hsvLow = np.array([0, 0, 218])
hsvUpp = np.array([157, 54, 255])

dilateKernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))  
dilate_i= 9

minW= 35
minH= 35
maxW= 300
maxH= 300

while(True):
    ret, frame = video_capture.read()
    final = frame

    ##[+:]=== Use this to find the ideal value range, which changes depending on the lighting ===============================\\
    tick= clk//clkRate + 1
    cv2.putText(frame, str(tick), org=(10, 20), fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale=.5, color= bgrOrange, thickness=2)
    clk+= 1
    if clk>clkLim: clk=0
    ##=======================================================================================================================//


    #--option 1: use grayscale-----
    img2gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret, grayMask = cv2.threshold(img2gray, grayLow, grayUpp, cv2.THRESH_BINARY)
    imgFinal_gray = cv2.bitwise_and(img2gray, img2gray, mask=grayMask)

    #---Option 2: use hsv-----------
    img2hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsvMask = cv2.inRange(img2hsv, hsvLow, hsvUpp)
    
    # for black text , cv.THRESH_BINARY_INV
    ret, new_img = cv2.threshold(imgFinal_gray, threshLow, threshUpp, cv2.THRESH_BINARY)  
    ret, new_img_inv = cv2.threshold(imgFinal_gray, threshLow, threshUpp, cv2.THRESH_BINARY_INV)  

#[::+::]==========|============Debug===========|===========================|+|+|+|\\
    # clkRate= 21
    # erodeKernel= np.ones((tick,tick), np.uint8)
    # threshLow= 255-tick
    # dilateKernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (tick,tick))  
    # dilate_i= tick
#[\\+\\]==========|============================|===========================|-|-|-|//

    # dilate , more the iteration more the dilation
    dilated = cv2.dilate(new_img_inv, dilateKernel, iterations= dilate_i)  


    # for cv2.x.x: 
    # _, contours, hierarchy = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)  # findContours returns 3 variables for getting contours

    # for cv3.x.x  comment above line and uncomment line below:
    #image, contours, hierarchy = cv2.findContours(dilated,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

    # [::]-- Selection: 
    contours, hierarchy = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    for contour in contours:
        [x, y, w, h] = cv2.boundingRect(contour)

        # if w < minW and h < minH:
        #     continue
        # if w > maxW and h > maxH:
        #     continue

        if w<maxW and h<maxH and w>minW and h>minH:
            # draw rectangle around contour on original image
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 255), 2)
            cv2.putText(frame, '[' + str(w)+'x'+str(h)+']', org=(x+w//2, y+h//2), 
                        fontFace=cv2.FONT_HERSHEY_DUPLEX, 
                        fontScale=.5, color= bgrCyan, 
                        thickness=2)
        
 
    #cv2.imshow('0 original frame',frame)
    cv2.imshow('1 threshold', imgFinal_gray)
    cv2.imshow("1 grayMask", grayMask)
    # cv2.imshow("2  hsvMask", hsvMask)
    # cv2.imshow("3 White Text", new_img)
    cv2.imshow("4 Black Text", new_img_inv)
    cv2.imshow("5 dilated", dilated)
    cv2.imshow("6 final", final)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

## Rectangle + Trackbars (CVC)

In [1]:
import cv2
import numpy as np

frameWidth = 640
frameHeight = 480
cap = cv2.VideoCapture(0)
cap.set(3, frameWidth)
cap.set(4, frameHeight)

def nothing(x):
    pass

cv2.namedWindow("Trackbars")
cv2.resizeWindow("Trackbars", 640, 240)
cv2.createTrackbar("Threshold 1", "Trackbars", 14, 255, nothing)
cv2.createTrackbar("Threshold 2", "Trackbars", 23, 255, nothing)
cv2.createTrackbar("Area", "Trackbars", 0, 5000, nothing)
cv2.createTrackbar("maxW", "Trackbars", 1000, 2000, nothing)
cv2.createTrackbar("minW", "Trackbars", 0, 1000, nothing)
cv2.createTrackbar("maxH", "Trackbars", 1000, 2000, nothing)
cv2.createTrackbar("minH", "Trackbars", 0, 1000, nothing)

def stackImages(scale, imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width =imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range (0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape[:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0,0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y] = cv2.cvtColor(imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank] *rows
        for x in range (0,rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0,rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0,0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None, scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor = np.hstack(imgArray)
        ver = hor
    return ver
                    
def getContours(img, imgContour):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    #cv2.drawContours(imgContour, contours, -1, (255, 0, 255), 7)
    
    for cnt in contours:
        """
        area = cv2.contourArea(cnt)
        maxArea = cv2.getTrackbarPos("Area", "Trackbars")
        if area < maxArea:
            cv2.drawContours(imgContour, cnt, -1, (255, 0, 255), 7)
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
            #print(approx)
            x, y, w, h = cv2.boundingRect(approx)
            cv2.rectangle(imgContour, (x, y), (x + w, y + h), (0, 255, 0), 5)

            #cv2.putText(imgContour, "Points: " + str(len(approx)), (x + w + 20, y + 20), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(imgContour, "Area: " + str(int(area)), (x + w + 20, y + 45), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 255, 0), 2)
        """
        
        peri = cv2.arcLength(cnt, True)
        area = cv2.contourArea(cnt)
        approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
        x, y, w, h = cv2.boundingRect(approx)
        maxW = cv2.getTrackbarPos("maxW", "Trackbars")
        maxH = cv2.getTrackbarPos("maxH", "Trackbars")
        minW = cv2.getTrackbarPos("minW", "Trackbars")
        minH = cv2.getTrackbarPos("minH", "Trackbars")
        if w < maxW and h < maxH and w > minW and h > minH:
            cv2.rectangle(imgContour, (x, y), (x + w, y + h), (0, 255, 0), 5)
            #cv2.putText(imgContour, "Points: " + str(len(approx)), (x + w + 20, y + 20), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(imgContour, "Area: " + str(int(area)), (x + w + 20, y + 45), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 255, 0), 2)
        

    
while True:
    success, img = cap.read()
    imgContour = img.copy()

    imgBlur = cv2.GaussianBlur(img, (7,7), 1)
    imgGray = cv2.cvtColor(imgBlur, cv2.COLOR_BGR2GRAY)

    threshold1 = cv2.getTrackbarPos("Threshold 1", "Trackbars")
    #threshold1 = 14
    threshold2 = cv2.getTrackbarPos("Threshold 2", "Trackbars")
    #threshold2 = 23
    imgCanny = cv2.Canny(imgGray, threshold1, threshold2)
    #imgCanny = cv2.Canny(imgGray, 143, 102)
    kernel = np.ones((5,5))
    imgDil = cv2.dilate(imgCanny, kernel, iterations = 1)

    getContours(imgDil, imgContour)

    imgStack = stackImages(0.8, ([img, imgGray, imgCanny], [imgDil, imgContour, imgContour]))

    cv2.imshow("Result", imgStack)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

## Filters

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture(1)
while True:
    ret, capturing = cap.read()
    hsv = cv2.cvtColor(capturing, cv2.COLOR_BGR2HSV)
    blur = cv2.GaussianBlur(hsv, (7, 7), 0)
    edge = cv2.Canny(blur, 100, 100)

    # Make it in a way that only shows the shapes and blacks out everything else
    ret, thresh1 = cv2.threshold(edge, 220, 255, cv2.THRESH_BINARY)
    cv2.imshow('Original', capturing)
    cv2.imshow('Edges', thresh1)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

##  __[:+:] Tasks__
- [x] Gate Task Circle Detection 1
- [x] Gate Task Circle Detection 2
- [ ] OCR
- [ ] Morphological Gradient; gradient = cv.morphologyEx(img, cv.MORPH_GRADIENT, kernel)
- [ ] Trimmed Mean
- [ ] Box Detection
- [ ] Ensemble of Algorithms
    - [ ] Helper Functions
    - [ ] Polling System
